## Unificação completa das bases (prefeitos 2024, 2k-200k habitantes)

In [ ]:
# Passo 1: gerar CSV de emendas com partidos
import subprocess
subprocess.run([
    'python', '../src/insert_parties.py',
    '--emendas', '../data/emendas_por_favorecido.csv',
    '--mapping', '../json_data/politicos_partidos_padronizado.json',
    '--out', '../data/emendas_por_favorecido_partidos.csv'
], check=True)

CompletedProcess(args=['python', '../src/insert_parties.py', '--emendas', '../data/emendas_por_favorecido.csv', '--mapping', '../json_data/politicos_partidos_padronizado.json', '--out', '../data/emendas_por_favorecido_partidos.csv'], returncode=0)

In [ ]:
import pandas as pd
import re
from unidecode import unidecode

def normalize_municipio(nome: str) -> str:
    if pd.isna(nome):
        return None
    nome = unidecode(str(nome)).upper().strip()
    m = re.search(r'(\(\w{2}\))$', nome)
    if m:
        uf = m.group(1)[1:-1]
        base = nome[:m.start()].strip()
    else:
        m = re.search(r'\s-\s([A-Z]{2})$', nome)
        if m:
            uf = m.group(1)
            base = nome[:m.start()].strip()
        else:
            uf = None
            base = nome
    return f"{base} - {uf}" if uf else base

# Emendas agregadas por município e partido
emendas = pd.read_csv('../data/emendas_por_favorecido_partidos.csv', encoding='utf-8-sig')
emendas['municipio'] = (emendas['Município Favorecido'] + ' - ' + emendas['UF Favorecido']).apply(normalize_municipio)
agg_pix_party = (emendas.groupby(['municipio','siglaPartido'], as_index=False)['Valor Recebido']
                        .sum()
                        .rename(columns={'Valor Recebido':'valor_pix'}))

# Indicadores IBGE
dens = pd.read_csv('../data/densidade.csv')
dens['municipio'] = dens['name_muni_x'].apply(normalize_municipio)
dens = dens[['code_muni','municipio','population_2022','densidade_demografica']]

pib = pd.read_csv('../data/pib_per_capita.csv')
pib['municipio'] = pib['name_muni'].apply(normalize_municipio)
pib = pib[['municipio','pib_per_capita_2021']]

idhm = pd.read_csv('../data/idhm.csv')
idhm['municipio'] = idhm['Territorialidades'].apply(normalize_municipio)
idhm['alfabetizacao_2010'] = 100 - idhm['Taxa de analfabetismo - 15 anos ou mais de idade 2010']
idhm = idhm[['municipio','IDHM 2010','alfabetizacao_2010']]

# Resultados eleitorais
results = pd.read_csv('../data/resultados_eleicoes.csv')
results['municipio'] = (results['NM_MUNICIPIO'] + ' - ' + results['SG_UF']).apply(normalize_municipio)
agg = results.groupby(['municipio','NM_URNA_CANDIDATO','SG_PARTIDO','DS_SIT_TOT_TURNO'], as_index=False)['QT_VOTOS_NOMINAIS_VALIDOS'].sum()

tot = agg.groupby('municipio')['QT_VOTOS_NOMINAIS_VALIDOS'].sum()
agg = agg.join(tot, on='municipio', rsuffix='_tot')
agg['pct_votos'] = agg['QT_VOTOS_NOMINAIS_VALIDOS'] / agg['QT_VOTOS_NOMINAIS_VALIDOS_tot']

winners = agg[agg['DS_SIT_TOT_TURNO'] == 'ELEITO']

# Unificação das bases
base = winners.merge(dens, on='municipio', how='left')
base = base.merge(agg_pix_party, left_on=['municipio','SG_PARTIDO'], right_on=['municipio','siglaPartido'], how='left')
base = base.merge(pib, on='municipio', how='left')
base = base.merge(idhm, on='municipio', how='left')

base['emendas_pix_per_capita_partido_prefeito_eleito'] = base['valor_pix'] / base['population_2022']
base['emendas_pix_per_capita_partido_prefeito_eleito'] = base['emendas_pix_per_capita_partido_prefeito_eleito'].fillna(0)
base['sigla_municipio'] = base['municipio'].str.extract(r'-\s*(\w{2})$')[0]

base_filtered = base[(base['population_2022'] >= 2000) & (base['population_2022'] <= 200000)]
final = base_filtered[['municipio','sigla_municipio','NM_URNA_CANDIDATO','SG_PARTIDO','pct_votos','emendas_pix_per_capita_partido_prefeito_eleito','IDHM 2010','alfabetizacao_2010','pib_per_capita_2021','densidade_demografica']]
final = final.rename(columns={'NM_URNA_CANDIDATO':'prefeito_eleito_2024',
                              'SG_PARTIDO':'sigla_partido_prefeito_eleito',
                              'pct_votos':'porcentagem_votos_validos_2024',
                              'IDHM 2010':'idhm_2010',
                              'densidade_demografica':'densidade_demografica_2010'})

final.to_csv('../data/dados_unificados_prefeitos_200k.csv', index=False, encoding='utf-8-sig')
final.head()
